In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [46]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [47]:
# add your code here - consider creating a new cell for each section of code
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

valid_users = user_counts[user_counts >= 200].index
valid_books = book_counts[book_counts >= 100].index

df_filtered = df_ratings[
    df_ratings['user'].isin(valid_users) &
    df_ratings['isbn'].isin(valid_books)
]

df = df_filtered.merge(df_books, on="isbn")

df = df.drop_duplicates(['user', 'title'])

book_user_matrix = df.pivot_table(
    index='title',
    columns='user',
    values='rating'
).fillna(0)

In [48]:
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(book_user_matrix.values)
book_titles = book_user_matrix.index.tolist()

In [49]:
# function to return recommended books - this will be tested
def get_recommends(book_title = ""):
  if book_title not in book_user_matrix.index:
        return []

  book_idx = book_titles.index(book_title)
  book_vector = book_user_matrix.iloc[book_idx].values.reshape(1, -1)

  distances, indices = model.kneighbors(book_vector, n_neighbors=6)

  recommends = []

  for i in range(5, 0, -1):
    idx = indices[0][i]
    recommends.append([book_user_matrix.index[idx], distances[0][i]])

  return [book_title, recommends]

In [50]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float32(0.8016211)], ['The Weight of Water', np.float32(0.77085835)], ['The Surgeon', np.float32(0.7699411)], ['I Know This Much Is True', np.float32(0.7677075)], ['The Lovely Bones: A Novel', np.float32(0.7234864)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
